In [1]:
# Import Libraries
try:
    # PYSPARK
    from pyspark.sql import SparkSession
    from pyspark import SparkContext
    from pyspark.sql import SQLContext
    from pyspark.sql import DataFrame
    import pyspark.sql.types as tp
    import pyspark.sql.functions as F
    
    # OTHER LIBRARIES
    import pandas as pd
    import numpy as np
    import glob
    from functools import reduce
    import seaborn as sns
    import matplotlib.pyplot as plt
    from urllib.request import urlopen
    import datetime
    from pathlib import Path
    
    print('[SUCCESS]')

    #CATCH ERROR IMPORTING A LIBRARY
except ImportError as ie:
    raise ImportError(f'[Error importing]: {ie}')

[SUCCESS]


In [2]:
# INITIALIZING OUR SPARK SESSION AND APP
# - this will be the name for the spark session when running our individual notebooks
spark = SparkSession.builder.master('local').config("spark.executor.memory", "1g").config("spark.driver.memory", "2g").appName('UsedCar_Project').getOrCreate()

In [3]:
# Best Way to read in the data since file is to big to upload to Git
# Copy and paste this cell into your individual notebooks and run to load data


def load_data():
    path = Path('car_prices.csv') # CHECKS TO SEE IF FILE EXIST
    
    # IF FILE DOES NOT EXIST
    ## DOWNLOAD DATA FROM GOOGLE STORAGE
    if not path.is_file():
        !wget https://storage.googleapis.com/iamangelsh-public-datasets/car_prices.csv # THIS COMMAND WILL ONLY RUN IN DOCKER CONTAINER USING JUPYTER NOTEBOOK
    
    df = spark.read.option("header",True).csv("car_prices.csv") # LOAD IN DATA WITH HEADERS
    
    return df # RETURN SPARK DATAFRAME


# LOAD THE DATA
df = load_data()

# SHOW DATA
df.show(5)

print(f'Number of columns: {len(df.columns)} \nNumber of Rows: {df.count()}')

--2022-04-04 22:22:19--  https://storage.googleapis.com/iamangelsh-public-datasets/car_prices.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89088424 (85M) [application/vnd.ms-excel]
Saving to: ‘car_prices.csv’

car_prices.csv      100%[===================>]  84.96M  4.23MB/s    in 19s     

2022-04-04 22:22:39 (4.39 MB/s) - ‘car_prices.csv’ saved [89088424/89088424]

+----+-----+-------------------+----------+-----+------------+-----------------+-----+---------+--------+-----+--------+--------------------+-----+------------+--------------------+
|year| make|              model|      trim| body|transmission|              vin|state|condition|odometer|color|interior|              seller|  mmr|sellingprice|            saledate|
+----+-----+-------------------+----------+

# Analysis Below

In [6]:
df.head()

Row(year='2015', make='Kia', model='Sorento', trim='LX', body='SUV', transmission='automatic', vin='5xyktca69fg566472', state='ca', condition='5', odometer='16639', color='white', interior='black', seller='kia motors america, inc', mmr='20500', sellingprice='21500', saledate='Tue Dec 16 2014 12:30:00 GMT-0800 (PST)')